In [18]:
from sklearn import metrics, model_selection, ensemble
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
import xgboost as xgb
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import xgboost

In [4]:
try: 
    import optuna
except:
    !pip install optuna
    import optuna

In [12]:
train = pd.read_csv("../data/train_for_crossval.csv")


In [9]:
train, t = train_test_split(train, test_size=0.9)

In [15]:
X = train.drop(["target"], axis=1)
y = train["target"] * 100

In [16]:
num_train = train.select_dtypes([int, float])
cat_train = train.select_dtypes(object)

num = list(num_train.drop(['target'],axis=1))
cat = list(cat_train)

In [19]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import OrdinalEncoder

rmse = make_scorer(mean_squared_error, squared=False)

pipeline_num = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('normal', PowerTransformer()),
    ('scaling', StandardScaler()),       
])
pipeline_cat = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('encoding', OrdinalEncoder()),
    ('encoding', OneHotEncoder(handle_unknown='ignore')),
])
preprocessor = ColumnTransformer(n_jobs=-1,
    transformers=[
        ('num', pipeline_num, num),
        ('cat', pipeline_cat, cat),
        ], remainder="passthrough")

transformed = preprocessor.fit_transform(X)

# svd = TruncatedSVD(n_components=20, n_iter=7, random_state=42)
# transformed = svd.fit_transform(transformed)

def objective(trial):
    
    param_model = {
        # 'tree_method':'gpu_hist',
        'eval_metric':trial.suggest_categorical("eval_metric", ['rmse']),
        'n_estimators': trial.suggest_categorical("n_estimators", [500]),
        'random_state': trial.suggest_categorical("random_state", [0, 42]), 
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 0.9),
        'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1),
        'max_depth': trial.suggest_int('max_depth', 4, 6),
        'objective': trial.suggest_categorical("objective", ['reg:squarederror']),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-2, 100),
        'reg_lambda': trial.suggest_int('reg_lambda', 1, 100), 
        'subsample': trial.suggest_float('subsample', 0.5, 0.9)
    }
    
    pipeline_ridge = Pipeline(steps=[
                                    #  ('preprocessor', preprocessor), 
#                                      ('svd', TruncatedSVD(random_state=42, **param_svd)), 
                                     ('model', xgboost.XGBRegressor(**param_model)),
                                     ])

    rmse_mean_cv = cross_val_score(pipeline_ridge, transformed, y, cv=5, scoring=rmse, n_jobs=-1)
    print(rmse_mean_cv)

    return rmse_mean_cv.mean()

In [13]:
study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=1)

[I 2021-09-02 16:34:31,897] A new study created in memory with name: no-name-741e1650-52e5-4923-a5f6-5fbc4953ca8b
[I 2021-09-02 16:35:14,175] Trial 0 finished with value: 82.92358174265148 and parameters: {'eval_metric': 'rmse', 'random_state': 42, 'colsample_bytree': 0.5604837633773141, 'learning_rate': 0.44414091997299965, 'max_depth': 6, 'objective': 'reg:squarederror', 'reg_alpha': 41.256352859724856, 'reg_lambda': 61, 'subsample': 0.6539690494738835}. Best is trial 0 with value: 82.92358174265148.


[82.24299031 82.5774242  84.86366666 80.51329413 84.42053342]


In [14]:
best_params = study.best_trial.params
best_params

{'eval_metric': 'rmse',
 'random_state': 42,
 'colsample_bytree': 0.5604837633773141,
 'learning_rate': 0.44414091997299965,
 'max_depth': 6,
 'objective': 'reg:squarederror',
 'reg_alpha': 41.256352859724856,
 'reg_lambda': 61,
 'subsample': 0.6539690494738835}

# save configs

In [15]:
import yaml
import os


directory = './Configs'
if not os.path.exists(directory):
    os.makedirs(directory)
    
    
with open('./Configs/params_model_xgb.yaml', 'w') as outfile:
    yaml.dump(best_params, outfile, default_flow_style=False)